# 1. Audit the uncleaned data of osm_samples.db.

In [1]:
#Connect to osm_samples.db using sqlite3.
#First, query what are the keys of tags.
import sqlite3

db = sqlite3.connect("C:\\sqlite_windows\\osm_samples.db")
c = db.cursor()

In [2]:
#query about postcodes.
uncleaned_postcode_query = "select tags.value, count(*) as count \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(uncleaned_postcode_query)
uncleaned_postcodes = c.fetchall()
for row in uncleaned_postcodes:
    print(row)

('10314', 769)
('11234', 678)
('10312', 594)
('10306', 534)
('11236', 510)
('11706', 507)
('11385', 502)
('11746', 477)
('11743', 468)
('11757', 438)
('11229', 411)
('11203', 398)
('10469', 388)
('11207', 369)
('11434', 366)
('10305', 362)
('11208', 360)
('11223', 360)
('11704', 355)
('11357', 346)
('11204', 345)
('11731', 342)
('10466', 328)
('11221', 325)
('11214', 324)
('11215', 323)
('10465', 319)
('11413', 319)
('11210', 315)
('11725', 315)
('11219', 314)
('10309', 311)
('11358', 307)
('11220', 306)
('11420', 306)
('10304', 302)
('11209', 300)
('11412', 300)
('11377', 298)
('11379', 296)
('11717', 292)
('10308', 290)
('11235', 284)
('10301', 283)
('10461', 283)
('11230', 281)
('11364', 280)
('11795', 280)
('11228', 279)
('11233', 274)
('11729', 271)
('11378', 267)
('11368', 262)
('11375', 261)
('11419', 260)
('11365', 250)
('11373', 250)
('11432', 250)
('11768', 248)
('11218', 246)
('11355', 241)
('11212', 230)
('11361', 230)
('11422', 227)
('11433', 223)
('11435', 221)
('11367', 

In [19]:
uncleaned_postcode_first2digit_query = "select substr(tags.value, 1, 2), count(*) as count \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'postcode' \
        group by substr(tags.value, 1, 2) \
        order by count desc;"
c.execute(uncleaned_postcode_first2digit_query)
uncleaned_postcodes_first2digit = c.fetchall()
for row in uncleaned_postcodes_first2digit:
    print(row)

('11', 26811)
('10', 9473)
('07', 68)
('08', 66)
('NY', 8)
('06', 5)
('32', 1)
('56', 1)


In [18]:
uncleaned_postcode_not_5digits_query = "select tags.value, count(*) as count \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'postcode' and tags.value not like '_____' \
        group by tags.value \
        order by count desc;"
c.execute(uncleaned_postcode_not_5digits_query)
uncleaned_postcodes_not_5digits = c.fetchall()
for row in uncleaned_postcodes_not_5digits:
    print(row)

('NY 10533', 6)
('08901-8541', 2)
('07501-210', 1)
('08854-8009', 1)
('08854-8035', 1)
('08854-8037', 1)
('08854-8039', 1)
('08854-8052', 1)
('08901-1004', 1)
('08901-1904', 1)
('08901-2867', 1)
('08901-8502', 1)
('08901-8524', 1)
('08901-8528', 1)
('08901-8539', 1)
('08901-8553', 1)
('10016-0122', 1)
('115422', 1)
('320', 1)
('56', 1)
('NY 10001', 1)
('NY 10026', 1)


- Most of postcodes start from 10 or 11. 
- And some of them start from 08 or 07 or 06 and they mean New Jersey or Connecticut. I used [these](http://www.geonames.org/postal-codes/postal-codes-us.html) [sites](https://en.wikipedia.org/wiki/New_York_metropolitan_area). Therefore I can infer that this New York OSM data has information about not just New York state, but New York Metropolitan area.
- There are 3 different types of postal codes. First, 5 digit code. Second, 5 digit code added by 4 digits connected by '-'. Third, 5 digit code started by 'NY '.I think that I can programmatically unify them. I'm going to use 5 digit code and zip+4 code both because zip+4 code is better at mailing([reference](https://smartystreets.com/articles/zip-4-code))
- There are 2 or 3 or 6 digit code and I think that it was input error. When cleaning the data before sending into DB, I'll ignore them.

In [7]:
#query about postal_codes.
uncleaned_postalcode_query = "select * \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'postal_code';"
c.execute(uncleaned_postalcode_query)
uncleaned_postalcodes = c.fetchall()
for row in uncleaned_postalcodes:
    print(row)

(11696141, 'postal_code', '07933', 'regular')
(11697677, 'postal_code', '07933', 'regular')
(462441978, 'postal_code', '06905', 'regular')


- Most of postal codes are in the key named 'postcode'. But there are 3 postal codes that are in the key named 'postal_code'. It is similar to the 'fax' and 'Fax' problem, and I'm going to alter the key 'postal_code' to 'postcode'.

In [3]:
#query about websites.
uncleaned_site_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(uncleaned_site_query)
uncleaned_sites = c.fetchall()
for row in uncleaned_sites:
    print(row)

('baileyarboretum.org',)
('bigtreebottles.com',)
('bk.com',)
('brookwoodatbayshore.com',)
('hsbc.com',)
('http://527madisonave.com/',)
('http://825thirdave.com/',)
('http://auttharosthainewyork.com',)
('http://bathenhaven.com/',)
('http://brooklynsandbox.com',)
('http://brooklynsocialbar.com/',)
('http://celticcorner.us/',)
('http://charno4.com/',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://doriangraynyc.com',)
('http://drambar.com/',)
('http://ezcesspoollongisland.com',)
('http://fkh.clarkschools.org/',)
('http://gomusicworks.com',)
('http://greenwichlocksmiths.com',)
('http://gsalehouse.com/',)
('http://jadisnyc.com/',)
('http://jerichoanimalhospital.com',)
('http://keyportlibrary.org/',)
('http://kfsmortgage.com',)
('http://landmarklivingston.com/',)
('http://lennysnyc.com/stores#lenny-s-55th-street',)
('http://lfstores.com/',)
('http://livinglotsnyc.org/lot/58276/',)
('http://location.ctownsupermarkets.com/ny/jamaica/41-003/',)
('http://lucasstamford.com/'

In [16]:
uncleaned_site_query = "select count(distinct tags.value) \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'website' and tags.value like 'http%' \
        order by tags.value;"
c.execute(uncleaned_site_query)
uncleaned_sites = c.fetchall()
for row in uncleaned_sites:
    print(row[0])

178


In [13]:
uncleaned_site_without_http_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'website' and tags.value not like 'http%' \
        order by tags.value;"
c.execute(uncleaned_site_without_http_query)
uncleaned_sites_without_http = c.fetchall()
for row in uncleaned_sites_without_http:
    print(row)

('baileyarboretum.org',)
('bigtreebottles.com',)
('bk.com',)
('brookwoodatbayshore.com',)
('hsbc.com',)
('learnandplayny.com',)
('nycgovparks.org',)
('subway.com',)
('usps.com',)
('www.breadsbakery.com',)
('www.creek.net',)
('www.dallasbbq.com',)
('www.efanyc.org',)
('www.mcstudios.com/',)
('www.naturino.com',)
('www.skyviewonthehudson.com/',)
('www.westcaldwellsaladworks.com',)


- There are three different forms of website addresses. First, URL started from 'http' or 'https' scheme. Second, domain name started from 'www'. Third, domain name without 'www'. I think that it is better to make all website addresses which don't start with 'http' or 'https' start with 'http://' for the consistency. And the other reason is that even if there are some sites which don't use 'www' subdomain, there isn't any websites which can be accessed without using http protocol. Referenced [these](https://en.wikipedia.org/wiki/URL) [sites](https://stackoverflow.com/questions/8951423/html-links-without-http-protocol).

In [4]:
#query about phones.
uncleaned_phone_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(uncleaned_phone_query)
uncleaned_phones = c.fetchall()
for row in uncleaned_phones:
    print(row)

('(201) 444-8111',)
('(201) 612-0756',)
('(212) 260-8015',)
('(212) 340-0863',)
('(212) 675-0550',)
('(212) 982-0362',)
('(212)228-5483',)
('(347) 799-2640',)
('(516) 431-5431',)
('(631) 499-8580',)
('(718) 229-7600',)
('(718) 230-3881',)
('(718) 321-6893',)
('(718) 369-0033',)
('(718) 381-5136',)
('(718) 488-7005',)
('(718) 623-9065',)
('(718) 677-5811',)
('(718) 821-8085',)
('(718) 855-9600',)
('(732) 264-7983',)
('(855) 417-1284',)
('(973) 256-7742',)
('(973) 310-3650',)
('(973) 564-9092',)
('+1 (201) 398-0944',)
('+1 (516) 801-6544',)
('+1 (718) 250-0200',)
('+1 (718) 387-9617',)
('+1 203 863 3000',)
('+1 205 9007007',)
('+1 212 2222954',)
('+1 212 242 6200',)
('+1 212 254 0006',)
('+1 212 353-3780',)
('+1 212 502 4609',)
('+1 212 663 1580',)
('+1 212 663 6004',)
('+1 212 665 9800',)
('+1 212 675-0342',)
('+1 212 690 4000',)
('+1 212 749 1111',)
('+1 212 799-9010',)
('+1 212 874 4000',)
('+1 212 9660290',)
('+1 212 989 8737',)
('+1 212-254-1675',)
('+1 516 783 86 60',)
('+1 646 415

- There are lots of different types of phone numbers in the result. I'm going to make them have ['(NPA) NXX-XXXX' format](https://en.wikipedia.org/wiki/National_conventions_for_writing_telephone_numbers) following the US national telephone writing convention, where NPA is the numbering plan area code and NXX-XXXX is the subscriber number.

In [5]:
#query about faxs.
uncleaned_fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(uncleaned_fax_query)
uncleaned_faxes = c.fetchall()
for row in uncleaned_faxes:
    print(row)

('+1 212 242 1278',)
('+1 212 678 6842',)
('+1 732 5432409',)
('+1 973 761 9193',)
('+1-212-268-0847',)
('718-268-7382',)
('718-361-2734',)


In [6]:
#query about Faxs.
uncleaned_Fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(uncleaned_Fax_query)
uncleaned_Faxes = c.fetchall()
for row in uncleaned_Faxes:
    print(row)

('732-548-8887',)


- 'Fax' and 'fax' are same but divided by capital or small letter and the results are like phone number's. I'm going to alter the key 'Fax' to 'fax' and make the numbers have the [same format](http://wiki.openstreetmap.org/wiki/Key:phone) as phone number's.

In [10]:
#query about cuisines.
cuisines_query = "select count(*) as num\
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'cuisine';"
c.execute(cuisines_query)
cuisines_number = c.fetchall()
print(cuisines_number[0][0])

112


In [8]:
#query about cusines.
uncleaned_cusine_query = "select * \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'cusine';"
c.execute(uncleaned_cusine_query)
uncleaned_cusines = c.fetchall()
for row in uncleaned_cusines:
    print(row)

(4528128545, 'cusine', 'thai', 'regular')


- It is also the similar key name problem but in this time it looked like that the similar key name was made by mistake. Most of cuisines are in the key named 'cuisine' and there is a single cuisine in the key named 'cusine'.

In [53]:
#query about opening-hours.
uncleaned_oh_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'opening_hours' \
        order by tags.value;"
c.execute(uncleaned_oh_query)
uncleaned_ohs = c.fetchall()
for row in uncleaned_ohs:
    print(row)

('11:00 AM to Close',)
('24/7',)
('5pm - 9pm (weekdays) , 2pm - 8pm ( weekend)',)
('772',)
('Mo-Fr 05:00-21:00, Su 06:30-21:00, Sa 06:30-21:00, PH 07:00-19:00',)
('Mo-Fr 05:30-21:30; Sa-Su 07:00-21:30',)
('Mo-Fr 05:30-22:00; Sa-Su 06:00-22:00',)
('Mo-Fr 07:00-19:00; Sa-Su 08:00-16:00',)
('Mo-Fr 07:00-20:00; Sa-Su 08:00-20:00',)
('Mo-Fr 07:00-21:00; Sa 08:00-20:00; Su 09:00-19:00',)
('Mo-Fr 07:30-18:00; Sa,Su 08:00-16:00',)
('Mo-Fr 07:30-19:00; Sa 07:30-16:00; Su 11:00-15:00',)
('Mo-Fr 08:00-18:00',)
('Mo-Fr 08:30-17:00; Sa 10:00-14:00',)
('Mo-Fr 08:30-18:00; Sa 09:00-15:00',)
('Mo-Fr 09:00-17:00',)
('Mo-Fr 09:00-20:00; Sa-Su 10:00-18:00',)
('Mo-Fr 10:00-19:30; Sa 10:00-18:00; Su 10:00-17:00',)
('Mo-Fr 10:00-20:00; Sa 10:00-18:00; Su 11:00-17:00',)
('Mo-Fr 11:00-22:00; Sa-Su 12:00-22:00',)
('Mo-Fr 12:00-16:30, 17:00-23:00; Sa-Su 12:00-23:30',)
('Mo-Fr 13:00-01:00; Sa-Su 12:00-01:00',)
('Mo-Fr 14:00-04:00',)
('Mo-Sa 07:00-19:00',)
('Mo-Sa 07:00-22:00',)
('Mo-Sa 08:00-04:00; Su 12:00-04:0

- There are some rules to write opening hours data in the [wiki](http://wiki.openstreetmap.org/wiki/Key:opening_hours). But we can find that there are some datas which don't follow the rules. In the results, some of them use 'am' or 'pm' instead of 'hh:mm' syntax. Also, some of them use words like 'until' and 'to' when it is possible to use '-' symbol. It can be a small problem when users read the datas, but it can be a big problem to classify them programmatically. It is a consistency matter.

In [30]:
#query about maxspeed.
uncleaned_maxspeed_query = "select distinct tags.value, tags.type \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'maxspeed' \
        order by tags.value;"
c.execute(uncleaned_maxspeed_query)
uncleaned_maxspeeds = c.fetchall()
for row in uncleaned_maxspeeds:
    print(row)

('10', 'regular')
('10 mph', 'regular')
('110 mph', 'regular')
('125 mph', 'regular')
('15', 'regular')
('15 mph', 'regular')
('160', 'regular')
('177', 'regular')
('20 mph', 'regular')
('201', 'regular')
('217', 'regular')
('25 mph', 'regular')
('30', 'regular')
('30 mph', 'regular')
('35', 'regular')
('35 mph', 'regular')
('40 mph', 'regular')
('45 mph', 'regular')
('48', 'regular')
('50', 'regular')
('50 mph', 'regular')
('55 mph', 'regular')
('60', 'regular')
('60 mph', 'regular')
('64', 'regular')
('65 mph', 'regular')
('70 mph', 'regular')
('72', 'regular')
('75 mph', 'regular')
('80', 'regular')
('90 mph', 'regular')
('Amtrak ETT 2010-01-18', 'source')
('Conrail ETT 2013-07-01', 'source')
('Conrail ETT 2013-10-17', 'source')
('signals', 'regular')


- Some of them are just number and others are number added by 'mph'. According to the [wiki](http://wiki.openstreetmap.org/wiki/Key:maxspeed), 'km/h(or kph)' is the basic unit of maxspeed. And the value which not measured by kph should be added to the end of it. 
- But when I see the numbers which aren't added by any unit, some of them are finished by '5' or '0', and others are finished by '1', '2', '7', etc. I think that it is reasonable to think that the former numbers' unit is 'mph', the [US standard speed unit](https://en.wikipedia.org/wiki/Miles_per_hour), and the latter numbers are converted to 'km/h' from 'mph'(because 48, 64, 72, 177, 201, 217 km/h can be converted to 30, 40, 45, 110, 125, 135 mph each). But it is my guess and to correctly add units, I need to find the maxspeed information for each place. Approximate speed limits in New York is in [here](https://en.wikipedia.org/wiki/Speed_limits_in_the_United_States_by_jurisdiction#New_York).
- And some values are about maxspeed's source, not about speed limit. They entered into here because when I process the keys which have lower colons in them, I divided them by ':' and the left part became types and the right part became keys. Therefore in this case of database, I need to use "where tags.key = 'maxspeed' and tags.type = 'regular'" in the query to extract only speed limit information.

In [58]:
db.close()

### - Finally, I decided to solve the problems in postcode, postal_code, website, phone, fax, and Fax fields.

# 2. Audit the cleaned and uncleaned data of osm_small_samples.db.

In [11]:
#Connect to osm_small_samples.db using sqlite3.
import sqlite3

small_db = sqlite3.connect("C:\\sqlite_windows\\osm_small_samples.db")
c = small_db.cursor()

In [12]:
#query about postcodes.
small_postcode_query = "select tags.value, count(*) as count \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(small_postcode_query)
small_postcodes = c.fetchall()
for row in small_postcodes:
    print(row)

('10314', 75)
('11234', 75)
('10312', 58)
('10306', 55)
('11706', 54)
('11743', 52)
('11236', 49)
('11385', 49)
('11223', 47)
('11757', 47)
('11207', 45)
('10469', 41)
('11746', 40)
('10465', 36)
('11208', 36)
('11357', 36)
('10304', 35)
('10305', 35)
('11219', 35)
('11203', 34)
('11229', 34)
('11413', 34)
('11704', 34)
('11204', 33)
('11220', 33)
('11434', 33)
('11731', 33)
('11221', 32)
('11420', 32)
('11214', 31)
('11364', 31)
('11375', 31)
('10461', 30)
('11412', 30)
('10309', 29)
('10466', 29)
('11210', 29)
('11233', 29)
('11358', 29)
('11215', 28)
('11228', 28)
('11235', 28)
('11373', 28)
('11378', 28)
('11379', 28)
('11725', 28)
('10308', 27)
('11209', 27)
('11218', 27)
('11377', 27)
('11717', 27)
('11729', 27)
('11365', 26)
('11419', 26)
('10301', 25)
('11230', 25)
('11355', 25)
('11701', 24)
('11361', 23)
('11421', 23)
('11435', 23)
('11768', 23)
('11795', 23)
('10310', 22)
('11216', 22)
('11367', 22)
('11368', 22)
('11432', 22)
('11691', 22)
('11702', 22)
('11105', 21)
('1142

In [13]:
#query about postcodes.
small_uncleaned_postcode_query = "select tags.value, count(*) as count \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(small_uncleaned_postcode_query)
small_uncleaned_postcodes = c.fetchall()
for row in small_uncleaned_postcodes:
    print(row)

('10314', 75)
('11234', 75)
('10312', 58)
('10306', 55)
('11706', 54)
('11743', 52)
('11236', 49)
('11385', 49)
('11223', 47)
('11757', 47)
('11207', 45)
('10469', 41)
('11746', 40)
('10465', 36)
('11208', 36)
('11357', 36)
('10304', 35)
('10305', 35)
('11219', 35)
('11203', 34)
('11229', 34)
('11413', 34)
('11704', 34)
('11204', 33)
('11220', 33)
('11434', 33)
('11731', 33)
('11221', 32)
('11420', 32)
('11214', 31)
('11364', 31)
('11375', 31)
('10461', 30)
('11412', 30)
('10309', 29)
('10466', 29)
('11210', 29)
('11233', 29)
('11358', 29)
('11215', 28)
('11228', 28)
('11235', 28)
('11373', 28)
('11378', 28)
('11379', 28)
('11725', 28)
('10308', 27)
('11209', 27)
('11218', 27)
('11377', 27)
('11717', 27)
('11729', 27)
('11365', 26)
('11419', 26)
('10301', 25)
('11230', 25)
('11355', 25)
('11701', 24)
('11361', 23)
('11421', 23)
('11435', 23)
('11768', 23)
('11795', 23)
('10310', 22)
('11216', 22)
('11367', 22)
('11368', 22)
('11432', 22)
('11691', 22)
('11702', 22)
('11105', 21)
('1142

In [14]:
#query about websites.
small_site_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(small_site_query)
small_sites = c.fetchall()
for row in small_sites:
    print(row)

('http://brookwoodatbayshore.com',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://location.ctownsupermarkets.com/ny/jamaica/41-003/',)
('http://maps.google.com/local_url?dq=mi+Ranchito+Restaurant&q=http://www.miranchitogrillandbar.com/&oi=miw&sa=X&ct=miw_link&cd=1&cad=homepage,cid:1706131537235801532&ei=54vDUaLkB6fRxAGmtYDgBA&s=ANYYN7kQV_99A5w0y593WxyjRp5MfpJp2Q',)
('http://web.mta.info/nyct/service/bus/qnsche.htm',)
('http://www.dallasbbq.com',)
('http://www.dreamhotels.com/hotels/new-york/new-york/dream-downtown/hotel-overview',)
('http://www.dustyrosevintage.com',)
('http://www.jjay.cuny.edu/',)
('http://www.marrakechhotelnyc.com/',)
('http://www.nellyspillanes.com',)
('http://www.nyctacos.com/',)
('http://www.stagneschurchnyc.org',)
('http://www.thebeekmanpub.com/',)
('http://www.themexsoft.com',)
('http://www.whitecastle.com/locations/358',)
('https://www.shakeshack.com',)


In [15]:
#query about websites.
small_uncleaned_site_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(small_uncleaned_site_query)
small_uncleaned_sites = c.fetchall()
for row in small_uncleaned_sites:
    print(row)

('brookwoodatbayshore.com',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://location.ctownsupermarkets.com/ny/jamaica/41-003/',)
('http://maps.google.com/local_url?dq=mi+Ranchito+Restaurant&q=http://www.miranchitogrillandbar.com/&oi=miw&sa=X&ct=miw_link&cd=1&cad=homepage,cid:1706131537235801532&ei=54vDUaLkB6fRxAGmtYDgBA&s=ANYYN7kQV_99A5w0y593WxyjRp5MfpJp2Q',)
('http://web.mta.info/nyct/service/bus/qnsche.htm',)
('http://www.dreamhotels.com/hotels/new-york/new-york/dream-downtown/hotel-overview',)
('http://www.dustyrosevintage.com',)
('http://www.jjay.cuny.edu/',)
('http://www.marrakechhotelnyc.com/',)
('http://www.nellyspillanes.com',)
('http://www.nyctacos.com/',)
('http://www.stagneschurchnyc.org',)
('http://www.thebeekmanpub.com/',)
('http://www.themexsoft.com',)
('http://www.whitecastle.com/locations/358',)
('https://www.shakeshack.com',)
('www.dallasbbq.com',)


In [16]:
#query about phones.
small_phone_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(small_phone_query)
small_phones = c.fetchall()
for row in small_phones:
    print(row)

('(205) 900-7007',)
('(212) 222-2954',)
('(212) 268-5876',)
('(212) 462-0001',)
('(212) 749-1111',)
('(212) 792-9673',)
('(212) 874-4000',)
('(718) 482-7088',)
('(718) 657-0722',)


In [17]:
#query about phones.
small_uncleaned_phone_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(small_uncleaned_phone_query)
small_uncleaned_phones = c.fetchall()
for row in small_uncleaned_phones:
    print(row)

('+1 205 9007007',)
('+1 212 2222954',)
('+1 212 749 1111',)
('+1 212 874 4000',)
('+1-212-268-5876',)
('+1-212.462.0001',)
('+1-718-657-0722',)
('+12127929673',)
('1 718 482 7088',)


In [18]:
#query about faxs.
small_fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(small_fax_query)
small_faxes = c.fetchall()
for row in small_faxes:
    print(row)

('(212) 268-0847',)
('(212) 678-6842',)


In [19]:
#query about faxs.
small_uncleaned_fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(small_uncleaned_fax_query)
small_uncleaned_faxes = c.fetchall()
for row in small_uncleaned_faxes:
    print(row)

('+1 212 678 6842',)
('+1-212-268-0847',)


In [20]:
#query about Faxs.
small_Fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(small_Fax_query)
small_Faxes = c.fetchall()
for row in small_Faxes:
    print(row)

In [21]:
#query about Faxs.
small_uncleaned_Fax_query = "select distinct tags.value \
        from (select * from uncleaned_nodes_tags \
                union all \
                select * from uncleaned_ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(small_uncleaned_Fax_query)
small_uncleaned_Faxes = c.fetchall()
for row in small_uncleaned_Faxes:
    print(row)

In [22]:
small_db.close()

### - Found that the cleaning code is now without problem. Now I can apply the cleaning code to New_York_sample file.

# 3. Audit the cleaned data of osm_samples.db.

In [23]:
#Connect to osm_samples.db using sqlite3.
import sqlite3

db = sqlite3.connect("C:\\sqlite_windows\\osm_samples.db")
c = db.cursor()

In [24]:
#query about postcodes.
postcode_query = "select tags.value, count(*) as count \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(postcode_query)
postcodes = c.fetchall()
for row in postcodes:
    print(row)

('10314', 769)
('11234', 678)
('10312', 594)
('10306', 534)
('11236', 510)
('11706', 507)
('11385', 502)
('11746', 477)
('11743', 468)
('11757', 438)
('11229', 411)
('11203', 398)
('10469', 388)
('11207', 369)
('11434', 366)
('10305', 362)
('11208', 360)
('11223', 360)
('11704', 355)
('11357', 346)
('11204', 345)
('11731', 342)
('10466', 328)
('11221', 325)
('11214', 324)
('11215', 323)
('10465', 319)
('11413', 319)
('11210', 315)
('11725', 315)
('11219', 314)
('10309', 311)
('11358', 307)
('11220', 306)
('11420', 306)
('10304', 302)
('11209', 300)
('11412', 300)
('11377', 298)
('11379', 296)
('11717', 292)
('10308', 290)
('11235', 284)
('10301', 283)
('10461', 283)
('11230', 281)
('11364', 280)
('11795', 280)
('11228', 279)
('11233', 274)
('11729', 271)
('11378', 267)
('11368', 262)
('11375', 261)
('11419', 260)
('11365', 250)
('11373', 250)
('11432', 250)
('11768', 248)
('11218', 246)
('11355', 241)
('11212', 230)
('11361', 230)
('11422', 227)
('11433', 223)
('11435', 221)
('11367', 

In [29]:
#query about postal_codes.
postalcode_query = "select tags.value, count(*) as count \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'postal_code' \
        group by tags.value \
        order by count desc;"
c.execute(postalcode_query)
postalcodes = c.fetchall()
for row in postalcodes:
    print(row)

- All postcodes follow the 5 digit code or zip+4 code(xxxxx-xxxx) form. There isn't anymore key named 'postal_code'.

In [25]:
#query about websites.
site_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(site_query)
sites = c.fetchall()
for row in sites:
    print(row)

('http://527madisonave.com/',)
('http://825thirdave.com/',)
('http://auttharosthainewyork.com',)
('http://baileyarboretum.org',)
('http://bathenhaven.com/',)
('http://bigtreebottles.com',)
('http://bk.com',)
('http://brooklynsandbox.com',)
('http://brooklynsocialbar.com/',)
('http://brookwoodatbayshore.com',)
('http://celticcorner.us/',)
('http://charno4.com/',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://doriangraynyc.com',)
('http://drambar.com/',)
('http://ezcesspoollongisland.com',)
('http://fkh.clarkschools.org/',)
('http://gomusicworks.com',)
('http://greenwichlocksmiths.com',)
('http://gsalehouse.com/',)
('http://hsbc.com',)
('http://jadisnyc.com/',)
('http://jerichoanimalhospital.com',)
('http://keyportlibrary.org/',)
('http://kfsmortgage.com',)
('http://landmarklivingston.com/',)
('http://learnandplayny.com',)
('http://lennysnyc.com/stores#lenny-s-55th-street',)
('http://lfstores.com/',)
('http://livinglotsnyc.org/lot/58276/',)
('http://location.ctowns

- All website addresses start with 'http' or 'https'.

In [26]:
#query about phones.
phone_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(phone_query)
phones = c.fetchall()
for row in phones:
    print(row)

('(201) 224-8444',)
('(201) 330-1416',)
('(201) 398-0944',)
('(201) 444-8111',)
('(201) 455-9056',)
('(201) 612-0756',)
('(201) 865-3987',)
('(203) 324-1800',)
('(203) 531-7200',)
('(203) 703-9080',)
('(203) 863-3000',)
('(205) 900-7007',)
('(212) 222-2954',)
('(212) 224-4646',)
('(212) 227-5975',)
('(212) 228-5483',)
('(212) 229-1600',)
('(212) 238-7897',)
('(212) 242-6200',)
('(212) 254-0006',)
('(212) 254-1675',)
('(212) 260-7289',)
('(212) 260-8015',)
('(212) 265-7250',)
('(212) 268-5876',)
('(212) 280-8779',)
('(212) 340-0863',)
('(212) 353-3780',)
('(212) 462-0001',)
('(212) 502-4609',)
('(212) 517-6590',)
('(212) 663-1580',)
('(212) 663-6004',)
('(212) 665-9800',)
('(212) 675-0342',)
('(212) 675-0550',)
('(212) 686-6525',)
('(212) 687-1786',)
('(212) 690-4000',)
('(212) 691-6067',)
('(212) 749-1111',)
('(212) 792-9673',)
('(212) 799-9010',)
('(212) 874-4000',)
('(212) 912-9770',)
('(212) 966-0290',)
('(212) 979-0900',)
('(212) 982-0362',)
('(212) 982-2121',)
('(212) 983-5389',)


In [27]:
#query about faxs.
fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(fax_query)
faxes = c.fetchall()
for row in faxes:
    print(row)

('(212) 242-1278',)
('(212) 268-0847',)
('(212) 678-6842',)
('(718) 268-7382',)
('(718) 361-2734',)
('(732) 543-2409',)
('(732) 548-8887',)
('(973) 761-9193',)


In [28]:
#query about Faxs.
Fax_query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(Fax_query)
Faxes = c.fetchall()
for row in Faxes:
    print(row)

- All phone numbers and fax numbers follows the (xxx) xxx-xxxx format. And the key 'Fax' was changed to 'fax'.

# 4. Data exploration.

### File sizes

In [8]:
import os
osm_path = 'C:\\projects\\New_York_OSM\\New_York_sample.osm'
db_path = 'C:\\sqlite_windows\\osm_samples.db'
print('New_York_sample.osm :', int(round(os.stat(osm_path).st_size) / (1024.0 * 1024.0)), 'MB')
print('osm_samples.db :', int(round(os.stat(db_path).st_size) / (1024.0 * 1024.0)), 'MB')

basedir = 'C:\\projects\\New_York_OSM\\samples'
names = os.listdir(basedir)
paths = [os.path.join(basedir, name) for name in names[:5]]
sizes = [int(round(os.stat(path).st_size / (1024.0 * 1024.0))) for path in paths]
for i in range(5):
    print(names[i], ':', sizes[i], 'MB')

New_York_sample.osm : 92 MB
osm_samples.db : 102 MB
sample_nodes.csv : 33 MB
sample_nodes_tags.csv : 1 MB
sample_ways.csv : 4 MB
sample_ways_nodes.csv : 11 MB
sample_ways_tags.csv : 9 MB


In [2]:
#Connect to osm_samples.db using sqlite3.
import sqlite3

db = sqlite3.connect("C:\\sqlite_windows\\osm_samples.db")
c = db.cursor()

### Number of Nodes

In [3]:
num_nodes_query = "select count(*) from nodes"
c.execute(num_nodes_query)
num_nodes = c.fetchall()
print(num_nodes[0][0])

384694


### Number of Ways

In [4]:
num_ways_query = "select count(*) from ways"
c.execute(num_ways_query)
num_ways = c.fetchall()
print(num_ways[0][0])

60316


### Number of unique users

In [5]:
num_unique_users_query = "select count(distinct(e.uid)) \
                        from (select uid from nodes union all select uid from ways) e"
c.execute(num_unique_users_query)
num_unique_users = c.fetchall()
print(num_unique_users[0][0])

1931


### I'm going to explore about building, sport, and shop fields. And I'm going to query at nodes_tags and ways_tags tables. 
### Before I'm going to do that, I need to find whether there is any duplicate information in the tables related with the fields. I'm going to join nodes_tags, ways_tags and ways_nodes tables.  
 (Ways_nodes table is useful in this case because it has the information about which ways having which nodes.)

In [5]:
#query about id's dividness.
is_id_divided_query = "select nodes_tags.key, count(*) as num \
        from ways_tags \
            join ways_nodes on ways_tags.id = ways_nodes.id \
            join nodes_tags on ways_nodes.node_id = nodes_tags.id \
        where ways_tags.key = nodes_tags.key \
        group by nodes_tags.key \
        order by num desc;"
c.execute(is_id_divided_query)
is_id_divided = c.fetchall()
for row in is_id_divided:
    print(row)

('highway', 58)
('created_by', 35)
('power', 12)
('railway', 5)
('crossing', 4)
('ref', 4)
('name', 2)
('barrier', 1)


 - Building, sport and shop fields don't have duplicate information in the nodes_tags and ways_tags tables that I can unify the two tables without problem.

### Top 5 usage of buildings.

In [31]:
#query about building.
building_query = "select tags.value, count(*) as num \
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'building' and tags.value != 'yes' \
        group by tags.value \
        order by num desc \
        limit 5;"
c.execute(building_query)
buildings = c.fetchall()
for row in buildings:
    print(row)

('house', 5393)
('garage', 4088)
('shed', 489)
('commercial', 317)
('service', 218)


- Most of the buildings are for house or garage in New York.

### Top 5 sports facilities.

In [32]:
#query about sports.
sport_query = "select tags.value, count(*) as num \
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'sport' and tags.value != 'yes' \
        group by tags.value \
        order by num desc \
        limit 5;"
c.execute(sport_query)
sports = c.fetchall()
for row in sports:
    print(row)

('baseball', 67)
('tennis', 65)
('basketball', 35)
('soccer', 25)
('american_football', 11)


- 5 of the most common sports facilities in New York are for baseball, tennis, basketball, soccer and american football.

### The number of shop information.

In [37]:
#query about shop.
shop_query = "select tags.key, count(*) as num\
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'shop';"
c.execute(shop_query)
shops = c.fetchall()
for row in shops:
    print(row)

('shop', 211)


### The number of shop information which have phone number or email or website or facebook.

In [40]:
#query about shop.
shop_with_phone_query = "select tags.key, count(*) as num\
        from (select * from nodes_tags union all select * from ways_tags) tags \
        join \
        (select distinct(id) from nodes_tags where key in ('phone', 'email', 'website', 'facebook') \
            union all \
         select distinct(id) from ways_tags where key in ('phone', 'email', 'website', 'facebook')) ids \
        on tags.id = ids.id \
        where tags.key = 'shop';"
c.execute(shop_with_phone_query)
shop_with_phones = c.fetchall()
for row in shop_with_phones:
    print(row)

('shop', 40)


- The number of shop information in New York OSM sample is 211 but only 40 of them have contact information. I think that it is better to make each shop information to have at least one contact information in the map.